# Imports and Constants

An API key must be given. Registration required first.

In [71]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [72]:
%autoreload 2
import requests
import pandas as pd
import numpy as np

from datetime import datetime
from dateutil import parser

from dotenv import load_dotenv

from tqdm import tqdm, tqdm_notebook

# import metalhistory as mh
import metalhistory.data_query_functions as dqf

import requests
import pandas as pd
import xmltodict
import time

In [73]:
tqdm.pandas()

# For devs: Overview of functions
These functions from utils.py work (untested) so far:

In [74]:
# lastfm = dqf.LastFM()
# lastfm.get_album_info('Death', 'Symbolic', verbose=1)
# lastfm.get_album_matches('Burzum', 'Filosofem',  verbose=1)
# lastfm.get_track_info('Black Sabbath', 'Paranoid', 'War Pigs', verbose=1)

# Data Preprocessing

In [75]:
df_csv = pd.read_csv('data/MA_10k_albums.csv')
df_csv

,artist,album,MA_score
0,Slayer,Reign in Blood,36.01
1,Metallica,Kill 'Em All,33.39
2,Hades Archer,Penis Metal,32.67
3,Iron Maiden,Iron Maiden,32.38
4,Metallica,Master of Puppets,31.83
...,...,...,...
9995,Iron Maiden,Live at the Rainbow,1.92
9996,Jorn,Worldchanger,1.92
9997,Juggernaut,Trouble Within,1.92
9998,Lacrimas Profundere,Memorandum,1.92


## Collected all instances of an album

In [81]:
lastfm = dqf.LastFM()
results_df = pd.DataFrame()

LIMIT=10 # Obituary - Cause of Death is causing some error here at idx 38 URL special key probably causes that issue!

df_head = df_csv.head(LIMIT)

for idx, row in tqdm_notebook(df_head.iterrows(), total=df_head.shape[0], desc='artists'):
    artist = row['artist']
    album = row['album']
    
    print(idx,': Querying for', artist, '-', album)
    matches = list(lastfm.get_album_matches(album=album)['results']['albummatches'].values())[0]

    nr_matches =  len(matches)
    nr_kept_matches = 0

    for match in tqdm_notebook(matches, desc='album matches', leave=False):
        if match['artist'] == artist:
            results_df = results_df.append({
                'artist': artist,
                'album': album,
                'album_instance': match['name'],
                'lastfm_info': lastfm.get_album_info(artist=match['artist'], album=match['name'])
            }, ignore_index=True)
            nr_kept_matches += 1        
    print('Kept', nr_kept_matches, 'out of', nr_matches, 'matches for', artist, '-', album, end='\n\n')

artists:   0%|          | 0/10 [00:00<?, ?it/s]

0 : Querying for Slayer - Reign in Blood


album matches:   0%|          | 0/50 [00:00<?, ?it/s]

Kept 44 out of 50 matches for Slayer - Reign in Blood

1 : Querying for Metallica - Kill 'Em All


album matches:   0%|          | 0/50 [00:00<?, ?it/s]

...

Kept 11 out of 50 matches for Agalloch - The Mantle

8 : Querying for Metallica - Ride the Lightning


album matches:   0%|          | 0/50 [00:00<?, ?it/s]

Kept 40 out of 50 matches for Metallica - Ride the Lightning

9 : Querying for Black Sabbath - Paranoid


album matches:   0%|          | 0/50 [00:00<?, ?it/s]

Kept 17 out of 50 matches for Black Sabbath - Paranoid



Add some entries of the lastfm info as column to the dataframe.

In [82]:
def get_listeners(x):
    if isinstance(x, dict):
        return int(x['listeners'])
    else:
        return None

def get_playcounts(x):
    if isinstance(x, dict):
        return int(x['playcount'])
    else:
        return None


In [83]:
df = results_df
df['listeners'] = df.progress_apply(lambda row: get_listeners(row['lastfm_info']), axis=1)
df['playcount'] = df.progress_apply(lambda row: get_playcounts(row['lastfm_info']), axis=1)
df

100%|██████████| 321/321 [00:00<00:00, 85294.37it/s]


,album,album_instance,artist,lastfm_info,listeners,playcount
0,Reign in Blood,Reign in Blood,Slayer,"{'name': 'Reign in Blood', 'artist': 'Slayer',...",822203,15581366
1,Reign in Blood,Reign In Blood (Expanded),Slayer,"{'name': 'Reign In Blood (Expanded)', 'artist'...",165778,2129289
2,Reign in Blood,Reign In Blood (Expanded Edition),Slayer,"{'name': 'Reign In Blood (Expanded Edition)', ...",12823,372573
3,Reign in Blood,Reign In Blood (1994 Reissue),Slayer,"{'name': 'Reign In Blood (1994 Reissue)', 'art...",4856,130028
4,Reign in Blood,Reign In Blood (Remastered),Slayer,"{'name': 'Reign In Blood (Remastered)', 'artis...",2092,75517
...,...,...,...,...,...,...
316,Paranoid,"Paranoid (Deluxe Expanded Edition, 2cd, Sanctu...",Black Sabbath,"{'name': 'Paranoid (Deluxe Expanded Edition, 2...",1366,33895
317,Paranoid,"Paranoid [2009, Sanctuary, 1782444]",Black Sabbath,"{'name': 'Paranoid [2009, Sanctuary, 1782444]'...",977,26077
318,Paranoid,Paranoid - Black Box - The Complete Original ...,Black Sabbath,{'name': 'Paranoid - Black Box - The Complete...,1631,34951
319,Paranoid,"Paranoid [1989, Vertigo, 23PD-134]",Black Sabbath,"{'name': 'Paranoid [1989, Vertigo, 23PD-134]',...",1232,23582


Sum up the different entries that correspond to the same album:

In [84]:
#TODO: Should probably be part of the utils.py
cumulative_df = df.drop(['album_instance'], axis=1).groupby(['artist','album']).sum()
cumulative_df

listeners  playcount
artist        album                                        
Agalloch      The Mantle                  352918   12135266
Black Sabbath Paranoid                   1937861   21620006
Hades Archer  Penis Metal                    489       9519
Iron Maiden   Iron Maiden                8061742  137698026
              The Number of the Beast    1071179   15060514
Megadeth      Rust in Peace               767407   16167930
Metallica     Kill 'Em All                673096   16148307
              Master of Puppets          1231870   25630045
              Ride the Lightning         1163841   23615884
Slayer        Reign in Blood             1027432   18554025

## Get tags of the most relevant album match

In [87]:
#TODO: Move these functions to the data_query_functions.py !
def get_tags(x):
    tag_names = []
    for tag in x['tags']['tag']:
        tag_names.append(tag['name'])
    return tag_names

def get_url(x):
    return x['url']

def get_info(x):
    artist = x.name[0]
    album = x.name[1]
    lastfm = dqf.LastFM()
    album_info = lastfm.get_album_info(artist=artist, album=album)
    return album_info

def get_img(x):
    return x['image']

# TODO: Release date not present. Should use another API for that?
# def get_releasedate(x):
#     return x['releasedate']

def get_mbid(x):
    try:
        return x['mbid']
    except KeyError:
        return None

import requests
import pandas as pd
import xmltodict

def get_release_date(x):
    print(x)
    if x is not None:
        response = requests.get('http://musicbrainz.org/ws/2/release/' + str(x) + '?inc=release-groups&fmt=xml')
        while response.status_code == 503:
            print('503, wating for five seconds.')
            time.sleep(5)
            response = requests.get('http://musicbrainz.org/ws/2/release/' + str(x) + '?inc=release-groups&fmt=xml')

        response_dict = xmltodict.parse(response.text)

        return response_dict['metadata']['release']['release-group']['first-release-date']

    else:
        return None




In [88]:
cumulative_df['lastfm_info'] = cumulative_df.apply(lambda row: get_info(row), axis=1)
cumulative_df['tags'] = cumulative_df.apply(lambda row: get_tags(row['lastfm_info']), axis=1)
cumulative_df['url'] = cumulative_df.apply(lambda row: get_url(row['lastfm_info']), axis=1)
cumulative_df['images'] = cumulative_df.apply(lambda row: get_img(row['lastfm_info']), axis=1)
cumulative_df['mbid'] = cumulative_df.apply(lambda row: get_mbid(row['lastfm_info']), axis=1)
cumulative_df['release_date'] = cumulative_df.apply(lambda row: get_release_date(row['mbid']), axis=1)
cumulative_df.drop(['lastfm_info', 'url', 'images'], axis=1)


7ac99528-77a9-3624-84b7-3400f6f56e47
2982b682-36ea-3605-b959-04e746736070
None
25da813d-4dbd-32c0-aef0-307e790f0709
6ea83f20-d053-3495-bc25-76aa61da13ab
2b904e74-daba-397c-a151-bafb125ceb44
c06ed440-f25d-3127-aadb-ebe9c685b3d8
b30c90dd-83f1-4897-ae36-58fb3935e5ee
2236dd07-a2f3-466a-973d-9069001a89da
e80dcb10-f532-4da1-9458-afd541e4a9ea


listeners  playcount  \
artist        album                                           
Agalloch      The Mantle                  352918   12135266   
Black Sabbath Paranoid                   1937861   21620006   
Hades Archer  Penis Metal                    489       9519   
Iron Maiden   Iron Maiden                8061742  137698026   
              The Number of the Beast    1071179   15060514   
Megadeth      Rust in Peace               767407   16167930   
Metallica     Kill 'Em All                673096   16148307   
              Master of Puppets          1231870   25630045   
              Ride the Lightning         1163841   23615884   
Slayer        Reign in Blood             1027432   18554025   

                                                                                    tags  \
artist        album                                                                        
Agalloch      The Mantle               [folk metal, doom metal, black metal, albums I...   
Black Sabbath Paranoid                 [heavy metal, hard rock, albums I own, classic...   
Hades Archer  Penis Metal                                                             []   
Iron Maiden   Iron Maiden               [heavy metal, albums I own, NWOBHM, metal, 1980]   
              The Number of the Beast  [heavy metal, albums I own, NWOBHM, metal, Iro...   
Megadeth      Rust in Peace            [thrash metal, albums I own, speed metal, heav...   
Metallica     Kill 'Em All             [thrash metal, albums I own, metal, heavy meta...   
              Master of Puppets        [thrash metal, albums I own, metal, heavy meta...   
              Ride the Lightning       [thrash metal, albums I own, metal, heavy meta...   
Slayer        Reign in Blood           [thrash metal, albums I own, metal, speed meta...   

                                                                       mbid  \
artist        album                                                           
Agalloch      The Mantle               7ac99528-77a9-3624-84b7-3400f6f56e47   
Black Sabbath Paranoid                 2982b682-36ea-3605-b959-04e746736070   
Hades Archer  Penis Metal                                              None   
Iron Maiden   Iron Maiden              25da813d-4dbd-32c0-aef0-307e790f0709   
              The Number of the Beast  6ea83f20-d053-3495-bc25-76aa61da13ab   
Megadeth      Rust in Peace            2b904e74-daba-397c-a151-bafb125ceb44   
Metallica     Kill 'Em All             c06ed440-f25d-3127-aadb-ebe9c685b3d8   
              Master of Puppets        b30c90dd-83f1-4897-ae36-58fb3935e5ee   
              Ride the Lightning       2236dd07-a2f3-466a-973d-9069001a89da   
Slayer        Reign in Blood           e80dcb10-f532-4da1-9458-afd541e4a9ea   

                                      release_date  
artist        album                                 
Agalloch      The Mantle                2002-08-13  
Black Sabbath Paranoid                  1970-09-18  
Hades Archer  Penis Metal                     None  
Iron Maiden   Iron Maiden               1980-04-14  
              The Number of the Beast   1982-04-26  
Megadeth      Rust in Peace             1990-09-21  
Metallica     Kill 'Em All              1983-07-25  
              Master of Puppets               2003  
              Ride the Lightning        1984-07-30  
Slayer        Reign in Blood            1986-10-07

In [64]:
attr_list = ['release_date', 'listeners', 'playcount', 'tags', 'mbid', 'url', 'images']
cumulative_df[attr_list].sort_values(by='listeners', ascending=False).to_csv('./data/proc_MA_10k_albums.csv')